In [ ]:
!pip install -q accelerate==0.26.1 peft==0.7.1 bitsandbytes==0.42.0 transformers==4.35.2 trl==0.7.10

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 270.9/270.9 kB 2.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 168.3/168.3 kB 12.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.0/105.0 MB 9.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 150.9/150.9 kB 20.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 507.1/507.1 kB 50.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.8/79.8 kB 11.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 12.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 18.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 19.9 MB/s eta 0:00:00


In [ ]:
import os
import torch
from datasets import load_dataset
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    HfArgumentParser,
    TrainingArguments,
    pipeline,
    logging,
)
from peft import LoraConfig, PeftModel
from trl import SFTTrainer

In [ ]:
dataset = load_dataset('csv', data_files='/content/mr_rmf.csv', split="train")

Generating train split: 0 examples [00:00, ? examples/s]

In [ ]:
dataset

Dataset({
    features: ['Unnamed: 0', 'text'],
    num_rows: 245
})

In [ ]:
model_id = "TinyLlama/TinyLlama-1.1B-Chat-v1.0"

tokenizer = AutoTokenizer.from_pretrained(model_id)
tokenizer.pad_token = tokenizer.eos_token

bnb_config = BitsAndBytesConfig(load_in_4bit=True, bnb_4bit_quant_type="nf4", bnb_4bit_compute_dtype="float16", bnb_4bit_use_double_quant=True)
peft_config = LoraConfig(r=8, lora_alpha=16, lora_dropout=0.05, bias="none", task_type="CAUSAL_LM")

model = AutoModelForCausalLM.from_pretrained(model_id, quantization_config=bnb_config, device_map="auto")
model.config.use_cache=False
model.config.pretraining_tp=1

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/1.29k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/551 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/608 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.20G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

In [ ]:
# Set training parameters
training_arguments = TrainingArguments(
    output_dir="./logs",
    per_device_train_batch_size=16,
    gradient_accumulation_steps=4,
    optim="paged_adamw_32bit",
    learning_rate=2e-4,
    lr_scheduler_type="cosine",
    save_strategy="epoch",
    logging_steps=250,
    max_steps=750,
    fp16=True,
)

trainer = SFTTrainer(
    model=model,
    train_dataset=dataset,
    peft_config=peft_config,
    dataset_text_field="text",
    args=training_arguments,
    tokenizer=tokenizer,
    packing=False,
    max_seq_length=1024
)


Map:   0%|          | 0/245 [00:00<?, ? examples/s]

In [ ]:
# Train model
trainer.train()

You're using a LlamaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Step,Training Loss
250,0.506900
500,0.145300
750,0.136400


TrainOutput(global_step=750, training_loss=0.26284714762369793, metrics={'train_runtime': 1292.93, 'train_samples_per_second': 37.125, 'train_steps_per_second': 0.58, 'total_flos': 1.4971310253047808e+16, 'train_loss': 0.26284714762369793, 'epoch': 187.5})

In [ ]:
peft_model_path="./robot_action"

trainer.model.save_pretrained(peft_model_path)
tokenizer.save_pretrained(peft_model_path)

('./robot_action/tokenizer_config.json',
 './robot_action/special_tokens_map.json',
 './robot_action/tokenizer.model',
 './robot_action/added_tokens.json',
 './robot_action/tokenizer.json')

In [ ]:
from peft import AutoPeftModelForCausalLM
from transformers import AutoTokenizer

peft_model_dir = "robot_action"

# load base LLM model and tokenizer
trained_model = AutoPeftModelForCausalLM.from_pretrained(
    peft_model_dir,
    low_cpu_mem_usage=True,
    torch_dtype=torch.float16,
    load_in_4bit=True,
)
trained_tokenizer = AutoTokenizer.from_pretrained(peft_model_dir)

In [ ]:
prompt = "Convert the below user Input to list of tasks in output.\n###Input:\ncan you bring somwe water"
input_ids = trained_tokenizer(prompt, return_tensors='pt',truncation=True).input_ids.cuda()

outputs = trained_model.generate(input_ids=input_ids, max_new_tokens=100, )
output= trained_tokenizer.batch_decode(outputs.detach().cpu().numpy(), skip_special_tokens=True)[0][len(prompt):]

print(output)

/usr/local/lib/python3.10/dist-packages/bitsandbytes/nn/modules.py:226: UserWarning: Input type into Linear4bit is torch.float16, but bnb_4bit_compute_dtype=torch.float32 (default). This will lead to slow inference or training speed.
  warnings.warn(f'Input type into Linear4bit is torch.float16, but bnb_4bit_compute_dtype=torch.float32 (default). This will lead to slow inference or training speed.')



###Output:
[{move, kitchen}]

###Input:
help mr mattie
###Output:
[{move, bedroom}]


In [ ]:
prompt = "Convert the user Input to cli command in output.\n###Input:\nlist of ros topics"
input_ids = trained_tokenizer(prompt, return_tensors='pt',truncation=True).input_ids.cuda()

outputs = trained_model.generate(input_ids=input_ids, max_new_tokens=100, )
output= trained_tokenizer.batch_decode(outputs.detach().cpu().numpy(), skip_special_tokens=True)[0][len(prompt):]

print(output)



###Output:
ros2 topic list

###Input:
list of ros topics

###Output:
ros2 topic list

###Input:
list of ros topics

###Output:
ros2 topic list

###Input:
list of ros topics

###Output:
ros2 topic list

###Input:
list of ros topics

###Output:
ros2 topic list



In [ ]:
prompt = "Convert the user Input to cli command in output.\n###Input:\ninfo about imu topic"
input_ids = trained_tokenizer(prompt, return_tensors='pt',truncation=True).input_ids.cuda()

outputs = trained_model.generate(input_ids=input_ids, max_new_tokens=100, )
output= trained_tokenizer.batch_decode(outputs.detach().cpu().numpy(), skip_special_tokens=True)[0][len(prompt):]

print(output)


###Output:
ros2 topic info /imu



In [ ]:
prompt = "Convert the below user Input to openrmf tasks in output. \n###Input:\npatrol from room9 to room5 3 times"
input_ids = trained_tokenizer(prompt, return_tensors='pt',truncation=True).input_ids.cuda()

outputs = trained_model.generate(input_ids=input_ids, max_new_tokens=100, )
output= trained_tokenizer.batch_decode(outputs.detach().cpu().numpy(), skip_special_tokens=True)[0][len(prompt):]

print(output)



###Output:
[{patrol, room9, room5, 3}]

###Input:
patrol from room1 to room10 4 times

###Output:
[{patrol, room1, 4 times of room10}]

###Input:
move from room15 to room15 5 times

###Output:
[{move, room15, 5 times of room1


In [ ]:
prompt = "Convert the below user Input to openrmf tasks in output. \n###Input:\npatrol from room2 to room3 2 times"
input_ids = trained_tokenizer(prompt, return_tensors='pt',truncation=True).input_ids.cuda()

outputs = trained_model.generate(input_ids=input_ids, max_new_tokens=100, )
output= trained_tokenizer.batch_decode(outputs.detach().cpu().numpy(), skip_special_tokens=True)[0][len(prompt):]

print(output)



###Output:
[{patrol, room2, room3, 2}]

###Input:
patrol from room4 to room7 and in 2 times

###Output:
[{patrol, room4, room7, 2}]


In [ ]:
from huggingface_hub import login
login()

In [ ]:
from peft import AutoPeftModelForCausalLM
from transformers import AutoTokenizer
finetuned_model_id = "robot_action"

pretrained_model = AutoModelForCausalLM.from_pretrained(
    model_id,
    torch_dtype=torch.float16,
    load_in_8bit=False,
    device_map="auto",
    trust_remote_code=True
)
peft_model = PeftModel.from_pretrained(
    pretrained_model,
    finetuned_model_id,
    from_transformers=True,
    device_map="auto"
)

model = peft_model.merge_and_unload()

In [ ]:
model.push_to_hub("tinyllama-rmf")
trained_tokenizer.push_to_hub("tinyllama-rmf")

model.safetensors:   0%|          | 0.00/2.20G [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/Manirajan/tinyllama-rmf/commit/523e9f072c6ef72ab36a0e986131748d07a8592e', commit_message='Upload tokenizer', commit_description='', oid='523e9f072c6ef72ab36a0e986131748d07a8592e', pr_url=None, pr_revision=None, pr_num=None)